# Urban digital twin scenarios
For the spatial operations you need at least arcpy or shapely and pyproj installed.

In [12]:
from arcgis.features import FeatureSet, GeoAccessor
from codecarbon import track_emissions
from configparser import ConfigParser
import json
import logging
import os
import pandas as pd
import sqlite3 as sql
import warnings

In [13]:
def read_geojson_as_sdf(filepath: str, encoding: str='utf8'):
    """
    Reads a GeoJSON file as a FeatureSet.
    """
    with open(filepath, encoding=encoding) as in_stream:
        return FeatureSet.from_geojson(json.load(in_stream)).sdf
    
def read_sqlite_as_sdf(db_filepath: str, select_statement: str, x_column: str='longitude', y_column: str='latitude'):
    """
    Reads the data from a sqlite database into main memory using a SQL statement.
    """
    with sql.connect(db_filepath) as connection:
        df = pd.read_sql_query(select_statement, connection)
        return GeoAccessor.from_xy(df, x_column, y_column)

In [14]:
config = ConfigParser()
config.read('config.user')

['config.user']

In [15]:
@track_emissions(project_name='Urban Digital Twin Bonn', output_file='emissions.user', offline=True, country_iso_code='USA')
def urban_intersect():
    logger = logging.getLogger('codecarbon')
    logger.info('Load urban datasets...')
    districts_sdf = read_geojson_as_sdf(config['DEFAULT']['CityDistrictsFilePath'], encoding='cp1252')
    streets_sdf = read_geojson_as_sdf(config['DEFAULT']['CityStreetFilePath'])
    streets_sdf = streets_sdf[streets_sdf['area'].isna()]
    traffic_sdf = read_sqlite_as_sdf(config['DEFAULT']['TrafficFilePath'], 'SELECT * from agent_pos LIMIT 10;')
    logger.info('Urban datasets loaded.')
    
    logger.info('Intersecting traffic with city districts...')
    traffic_joined_districts_sdf = traffic_sdf.spatial.join(districts_sdf, how='inner', op='intersects')
    logger.info(f'{traffic_joined_districts_sdf.shape[0]} traffic locations have intersections with city districts.')
    
    warnings.filterwarnings('ignore')
    
    logger.info('Projecting city streets...')
    streets_sdf.spatial.project(25832, 'DHDN_To_WGS_1984_4_NTv2')
    logger.info(f'{streets_sdf.shape[0]} city streets were projected.')
    
    logger.info('Projecting traffic locations...')
    traffic_sdf.spatial.project(25832, 'DHDN_To_WGS_1984_4_NTv2')
    logger.info(f'{traffic_sdf.shape[0]} traffic locations were projected.')
    
    warnings.filterwarnings('default')
    
    logger.info('Constructing city streets buffer...')
    buffered_streets_sdf = streets_sdf.copy()
    buffered_streets_sdf.SHAPE = streets_sdf.SHAPE.geom.buffer(50)
    logger.info(f'{buffered_streets_sdf.shape[0]} city streets buffer were constructed.')
    
    logger.info('Intersecting traffic with city streets buffer...')
    traffic_joined_buffered_streets_sdf = traffic_sdf.spatial.join(buffered_streets_sdf, how='inner', op='intersects')
    logger.info(f'{traffic_joined_districts_sdf.shape[0]} traffic locations have intersections with city streets buffer.')
    
    # Join traffic with intersected streets buffer with original streets
    # Calculate distance between traffic location and corresponding streets
    logger.info('Calculating distances to city streets...')
    traffic_joined_streets_sdf = traffic_joined_buffered_streets_sdf.join(streets_sdf, how='left', on='OBJECTID', rsuffix='_street')
    traffic_joined_streets_sdf['distance_to'] = traffic_joined_streets_sdf.SHAPE.geom.distance_to(streets_sdf.SHAPE_street)
    logger.info(f'{traffic_joined_streets_sdf.shape[0]} distances to city streets were calculated.')

In [16]:
urban_intersect()

[codecarbon INFO @ 19:54:39] offline tracker init
[codecarbon INFO @ 19:54:39] [setup] RAM Tracking...
[codecarbon INFO @ 19:54:39] [setup] GPU Tracking...
[codecarbon INFO @ 19:54:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:54:39] [setup] CPU Tracking...
[codecarbon INFO @ 19:54:39] Tracking Intel CPU via Power Gadget
[codecarbon INFO @ 19:54:41] >>> Tracker's metadata:
[codecarbon INFO @ 19:54:41]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 19:54:41]   Python version: 3.9.16
[codecarbon INFO @ 19:54:41]   Available RAM : 31.775 GB
[codecarbon INFO @ 19:54:41]   CPU count: 24
[codecarbon INFO @ 19:54:41]   CPU model: 12th Gen Intel(R) Core(TM) i7-12800HX
[codecarbon INFO @ 19:54:41]   GPU count: 1
[codecarbon INFO @ 19:54:41]   GPU model: 1 x NVIDIA RTX A2000 8GB Laptop GPU
[codecarbon INFO @ 19:54:41] Load urban datasets...
[codecarbon INFO @ 19:54:56] Energy consumed for RAM : 0.000050 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 19:54:56

[codecarbon INFO @ 19:59:15] 0.002192 kWh of electricity used since the begining.
[codecarbon INFO @ 19:59:27] Energy consumed for RAM : 0.000750 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 19:59:27] Energy consumed for all GPUs : 0.000536 kWh. All GPUs Power : 8.278 W
[codecarbon INFO @ 19:59:29] Energy consumed for all CPUs : 0.001023 kWh. All CPUs Power : 15.061400000000003 W
[codecarbon INFO @ 19:59:30] 0.002308 kWh of electricity used since the begining.
[codecarbon INFO @ 19:59:42] Energy consumed for RAM : 0.000789 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 19:59:42] Energy consumed for all GPUs : 0.000563 kWh. All GPUs Power : 8.343 W
[codecarbon INFO @ 19:59:44] Energy consumed for all CPUs : 0.001075 kWh. All CPUs Power : 14.8976 W
[codecarbon INFO @ 19:59:45] 0.002426 kWh of electricity used since the begining.
[codecarbon INFO @ 19:59:57] Energy consumed for RAM : 0.000828 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 19:59:57] Energy c

[codecarbon INFO @ 20:04:31] 0.004738 kWh of electricity used since the begining.
[codecarbon INFO @ 20:04:43] Energy consumed for RAM : 0.001567 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:04:43] Energy consumed for all GPUs : 0.001112 kWh. All GPUs Power : 8.217 W
[codecarbon INFO @ 20:04:45] Energy consumed for all CPUs : 0.002181 kWh. All CPUs Power : 17.0191 W
[codecarbon INFO @ 20:04:46] 0.004860 kWh of electricity used since the begining.
[codecarbon INFO @ 20:04:58] Energy consumed for RAM : 0.001606 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:04:58] Energy consumed for all GPUs : 0.001140 kWh. All GPUs Power : 8.277000000000001 W
[codecarbon INFO @ 20:05:00] Energy consumed for all CPUs : 0.002236 kWh. All CPUs Power : 16.128090909090908 W
[codecarbon INFO @ 20:05:01] 0.004981 kWh of electricity used since the begining.
[codecarbon INFO @ 20:05:13] Energy consumed for RAM : 0.001645 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:05:

[codecarbon INFO @ 20:09:47] 0.007272 kWh of electricity used since the begining.
[codecarbon INFO @ 20:09:58] Energy consumed for RAM : 0.002381 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:09:59] Energy consumed for all GPUs : 0.001681 kWh. All GPUs Power : 8.25 W
[codecarbon INFO @ 20:10:01] Energy consumed for all CPUs : 0.003324 kWh. All CPUs Power : 14.659799999999999 W
[codecarbon INFO @ 20:10:02] 0.007387 kWh of electricity used since the begining.
[codecarbon INFO @ 20:10:14] Energy consumed for RAM : 0.002420 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:10:14] Energy consumed for all GPUs : 0.001709 kWh. All GPUs Power : 8.358 W
[codecarbon INFO @ 20:10:16] Energy consumed for all CPUs : 0.003383 kWh. All CPUs Power : 17.148400000000002 W
[codecarbon INFO @ 20:10:17] 0.007512 kWh of electricity used since the begining.
[codecarbon INFO @ 20:10:29] Energy consumed for RAM : 0.002459 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:10:29

[codecarbon INFO @ 20:15:02] Energy consumed for all CPUs : 0.004292 kWh. All CPUs Power : 14.645900000000001 W
[codecarbon INFO @ 20:15:03] 0.009642 kWh of electricity used since the begining.
[codecarbon INFO @ 20:15:14] Energy consumed for RAM : 0.003199 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:15:15] Energy consumed for all GPUs : 0.002215 kWh. All GPUs Power : 7.947000000000001 W
[codecarbon INFO @ 20:15:16] Energy consumed for all CPUs : 0.004341 kWh. All CPUs Power : 14.540899999999999 W
[codecarbon INFO @ 20:15:18] 0.009755 kWh of electricity used since the begining.
[codecarbon INFO @ 20:15:30] Energy consumed for RAM : 0.003239 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 20:15:30] Energy consumed for all GPUs : 0.002241 kWh. All GPUs Power : 7.767 W
[codecarbon INFO @ 20:15:31] Energy consumed for all CPUs : 0.004392 kWh. All CPUs Power : 14.800099999999997 W
[codecarbon INFO @ 20:15:33] 0.009871 kWh of electricity used since the begining.
[cod

...

[codecarbon INFO @ 21:17:57] Energy consumed for all GPUs : 0.007274 kWh. All GPUs Power : 5.056 W
[codecarbon INFO @ 21:17:59] Energy consumed for all CPUs : 0.015989 kWh. All CPUs Power : 11.417222222222225 W
[codecarbon INFO @ 21:18:00] 0.036110 kWh of electricity used since the begining.
[codecarbon INFO @ 21:18:12] Energy consumed for RAM : 0.012886 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:18:12] Energy consumed for all GPUs : 0.007291 kWh. All GPUs Power : 5.083000000000001 W
[codecarbon INFO @ 21:18:14] Energy consumed for all CPUs : 0.016036 kWh. All CPUs Power : 13.612111111111114 W
[codecarbon INFO @ 21:18:15] 0.036212 kWh of electricity used since the begining.
[codecarbon INFO @ 21:18:27] Energy consumed for RAM : 0.012925 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:18:27] Energy consumed for all GPUs : 0.007307 kWh. All GPUs Power : 5.071 W
[codecarbon INFO @ 21:18:29] Energy consumed for all CPUs : 0.016078 kWh. All CPUs Power : 12.38566

[codecarbon INFO @ 21:23:13] Energy consumed for RAM : 0.013657 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:23:13] Energy consumed for all GPUs : 0.007632 kWh. All GPUs Power : 5.017 W
[codecarbon INFO @ 21:23:15] Energy consumed for all CPUs : 0.016893 kWh. All CPUs Power : 13.217600000000001 W
[codecarbon INFO @ 21:23:16] 0.038182 kWh of electricity used since the begining.
[codecarbon INFO @ 21:23:28] Energy consumed for RAM : 0.013695 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:23:28] Energy consumed for all GPUs : 0.007649 kWh. All GPUs Power : 5.1450000000000005 W
[codecarbon INFO @ 21:23:30] Energy consumed for all CPUs : 0.016941 kWh. All CPUs Power : 14.404666666666666 W
[codecarbon INFO @ 21:23:31] 0.038285 kWh of electricity used since the begining.
[codecarbon INFO @ 21:23:43] Energy consumed for RAM : 0.013734 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:23:43] Energy consumed for all GPUs : 0.007666 kWh. All GPUs Power : 5.10

[codecarbon INFO @ 21:28:17] 0.040165 kWh of electricity used since the begining.
[codecarbon INFO @ 21:28:29] Energy consumed for RAM : 0.014470 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:28:29] Energy consumed for all GPUs : 0.007995 kWh. All GPUs Power : 5.687 W
[codecarbon INFO @ 21:28:31] Energy consumed for all CPUs : 0.017799 kWh. All CPUs Power : 12.373222222222223 W
[codecarbon INFO @ 21:28:33] 0.040264 kWh of electricity used since the begining.
[codecarbon INFO @ 21:28:44] Energy consumed for RAM : 0.014508 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:28:44] Energy consumed for all GPUs : 0.008012 kWh. All GPUs Power : 5.079000000000001 W
[codecarbon INFO @ 21:28:46] Energy consumed for all CPUs : 0.017841 kWh. All CPUs Power : 12.913124999999999 W
[codecarbon INFO @ 21:28:47] 0.040360 kWh of electricity used since the begining.
[codecarbon INFO @ 21:28:59] Energy consumed for RAM : 0.014547 kWh. RAM Power : 11.915722846984865 W
[codecarbon IN

[codecarbon INFO @ 21:33:15] Energy consumed for all GPUs : 0.008318 kWh. All GPUs Power : 5.151 W
[codecarbon INFO @ 21:33:16] Energy consumed for all CPUs : 0.018603 kWh. All CPUs Power : 12.659111111111109 W
[codecarbon INFO @ 21:33:17] 0.042132 kWh of electricity used since the begining.
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1d1bd2331c0>
[codecarbon INFO @ 21:33:25] 150401 city streets buffer were constructed.
[codecarbon INFO @ 21:33:25] Intersecting traffic with city streets buffer...
[codecarbon INFO @ 21:33:30] Energy consumed for RAM : 0.015252 kWh. RAM Power : 11.915722846984865 W
[codecarbon INFO @ 21:33:30] Energy consumed for all GPUs : 0.008336 kWh. All GPUs Power : 5.122 W
[codecarbon INFO @ 21:33:32] Energy consumed for all CPUs : 0.018650 kWh. All CPUs Power : 12.976444444444446 W
[codecarbon INFO @ 21:33:33] 0.042239 kWh of electricity used since the begining.
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1d1affcce80>


AttributeError: 'DataFrame' object has no attribute 'SHAPE_street'

In [57]:

streets_sdf

,@id,area,bicycle,highway,name,surface,OBJECTID,SHAPE
28,way/22880467,<NA>,<NA>,pedestrian,Takuplatz,<NA>,29,"{""rings"": [[[6.9158428, 50.9615675], [6.915798..."
137,way/36809213,<NA>,<NA>,no,Ubierring,<NA>,138,"{""rings"": [[[6.9658678, 50.9225633], [6.965856..."
138,way/36809214,<NA>,<NA>,no,Ubierring,<NA>,139,"{""rings"": [[[6.9650702, 50.9222487], [6.965081..."
154,way/40291091,<NA>,<NA>,no,Ubierring,<NA>,155,"{""rings"": [[[6.9663071, 50.9218984], [6.966307..."
155,way/41362803,<NA>,<NA>,no,Chlodwigplatz,<NA>,156,"{""rings"": [[[6.9602065, 50.9212555], [6.960215..."
...,...,...,...,...,...,...,...,...
151045,way/557097695,<NA>,<NA>,service,<NA>,<NA>,151046,"{""paths"": [[[7.0551821, 50.9670198], [7.055306..."
151046,way/557154755,<NA>,<NA>,path,<NA>,unpaved,151047,"{""paths"": [[[6.9486374, 50.9557695], [6.949021..."
151047,way/557154756,<NA>,<NA>,path,<NA>,unpaved,151048,"{""paths"": [[[6.9489955, 50.9560589], [6.948642..."
151048,way/557154757,<NA>,<NA>,tertiary,Merheimer Straße,<NA>,151049,"{""paths"": [[[6.9496619, 50.9584811], [6.949690..."


In [77]:
import shapely

ModuleNotFoundError: No module named 'shapely'